In [57]:
root = '/home/gbischof/dama/fio_weka/'
files = ['nsls2_bm_nossd']
#files = ['garrett_all_jobs', 'garrett_all_jobs_12nodes', 
 #        'nsls2_bm_nossd', 'nsls2_bm_nossd2', 'nsls2_bm_ssd',
  #       'nsls2_CENTRAL_nossd', 'nsls2_CENTRAL_ssd']

In [58]:
from collections import defaultdict
import json

def parse_benchmark_files(root, files):
    results = {}
    for file in files:
        results[file] = parse_benchmark_file(root + file)
    return results

def get_next_section(file):
    section = []
    start = False
    
    for line in file:
        if start:
            section.append(line.replace("'", '"'))
        if line.find("Starting tests on") != -1:
            start = True
            n = line.split(' ')[3]
        if line.find("Tests complete.") != -1:
            return section, n
    return False, 0
    
def parse_benchmark_file(file):
    results = defaultdict(lambda: list())
    with open(file) as f:
        section, n = get_next_section(f)
        while section:
            # Parse the section.
            result = parse_section(section)
            
            # Update the results.
            results['n'].append(n)
            for key, value in result.items():
                results[key].append(value)
            
            # Get the next section of the file.
            section, n = get_next_section(f)      
    return dict(results)

def parse_section(section):
    iops = {**json.loads(section[25]), **json.loads(section[26])}
    iops = {'iops_' + key: value for key, value in iops.items()}
    assert iops.keys() == set(['iops_total_read_bandwidth', 'iops_avg_read_bandwidth', 'iops_total_bandwidth',
     'iops_avg_bandwidth', 'iops_avg_write_bandwidth', 'iops_total_write_bandwidth', 
     'iops_total write iops', 'iops_total iops', 'iops_avg write iops',
     'iops_total read iops', 'iops_avg read iops', 'iops_avg iops'])
    
    bw = {**json.loads(section[5]), **json.loads(section[6])}
    bw = {'bw_' + key: value for key, value in bw.items()}
    assert bw.keys() == set(['bw_total_read_bandwidth', 'bw_avg_read_bandwidth', 'bw_total_bandwidth',
     'bw_avg_bandwidth', 'bw_avg_write_bandwidth', 'bw_total_write_bandwidth',
     'bw_total write iops', 'bw_total iops', 'bw_avg write iops',
     'bw_total read iops', 'bw_avg read iops', 'bw_avg iops'])
    
    latency = json.loads(section[31])
    latency = {'latency_' + key: value for key, value in latency.items()}
    assert latency.keys() == set(['latency_write latency', 'latency_read latency'])
    
    write = {**json.loads(section[18]), **json.loads(section[19]), **json.loads(section[20])}
    write = {'detwrite_' + key: value for key, value in write.items()}
    assert write.keys() == set(['detwrite_total_read_bandwidth', 'detwrite_avg_read_bandwidth', 'detwrite_total_bandwidth', 
     'detwrite_avg_bandwidth', 'detwrite_avg_write_bandwidth', 'detwrite_total_write_bandwidth',
     'detwrite_total write iops', 'detwrite_total iops', 'detwrite_avg write iops',
     'detwrite_total read iops', 'detwrite_avg read iops',
     'detwrite_avg iops', 'detwrite_write latency', 'detwrite_read latency'])
    
    rw = {**json.loads(section[11]), **json.loads(section[12]), **json.loads(section[13])}
    rw = {'detrw_' + key: value for key, value in rw.items()}
    assert rw.keys() == set(['detrw_total_read_bandwidth', 'detrw_avg_read_bandwidth', 
     'detrw_total_bandwidth', 'detrw_avg_bandwidth', 'detrw_avg_write_bandwidth',
     'detrw_total_write_bandwidth', 'detrw_total write iops', 'detrw_total iops',
     'detrw_avg write iops', 'detrw_total read iops', 'detrw_avg read iops',
     'detrw_avg iops', 'detrw_write latency', 'detrw_read latency'])

    return {**iops, **bw, **latency, **write, **rw}

In [59]:
results = parse_benchmark_files(root, files)
print(results)

JSONDecodeError: Expecting value: line 2 column 1 (char 1)